In [1]:
require('nn')
require('hdf5')
require('optim')

In [2]:
f = hdf5.open("data.hdf5", "r")
X_train = f:read("train_input"):all()
Y_train = f:read("train_output"):all()
X_valid = f:read("valid_input"):all()
q_valid = f:read("valid_q"):all()
X_test = f:read("test_input"):all()
q_test = f:read("test_q"):all()
kaggle_valid = f:read("valid_kaggle"):all()
nclasses = f:read('nclasses'):all():long()[1]
nfeatures = f:read('nfeatures'):all():long()[1]
f:close()

window_size = X_train:size(2)
n_actual = nfeatures

In [4]:
--changes index to account for window size
function changeIndex(X, window_size, nfeatures)
    for i=1,X:size(2) do
        X:select(2,i):add((i-1)*nfeatures)
    end
    return X
end

function lookup(X, Y, win_size, C, K)
    W = torch.zeros(C, win_size*(K))
    for i=1,X:size(1) do
        for j=1,X:size(2) do
            local row = Y[i]
            local col = X[i][j]
            W[row][col] = W[row][col] + 1
        end
    end
    return W
end

In [49]:
function CBM(fc, Y, alpha, V)
    --calculate log posterior
    local cc = torch.zeros(1, V)
    local clp = torch.zeros(1, V)

    --collapse column wise so Nx1
    local hist = torch.histc(Y:double(), 10001)
    cc:add(torch.reshape(hist, 1, 10001))
    --THIS IS ALPHA
    --smoothing
    fc:add(alpha)
    local scc = fc:sum(2)
    
    fc:log()
    scc:log()
    
    scc:expand(scc, scc:size(1), fc:size(2))
    
    --calculate log prior
    --collapse down so 1x1
    local total = cc:sum(2)
    cc:div(total[1][1])
    cc:log()
    --clp:csub(math.log(Y:size(2)))
    
    return fc:csub(scc), cc
end

function predict(X, W, b)
  local jll = torch.zeros(X:size(1), W:size(1))
  for i = 1, X:size(1) do
    jll[i] = W:index(2, X[i]):sum(2)
    jll[i]:add(b)
  end
  return jll
end

In [47]:
function perplexity(jll, Y)
    local matrix = torch.zeros(Y:size(1), Y:size(2))
    for i=1, Y:size(1) do
        for j=1, Y:size(2) do
            idx = Y[i][j]
            matrix[i][j] = (jll[i][idx] * -1)
        end
    end
    return matrix
end

In [7]:
X_train = changeIndex(X_train, 5, nfeatures)
X_valid = changeIndex(X_valid, 5, nfeatures)

In [8]:
W = lookup(X_train, Y_train, window_size, 10001, nfeatures)

In [9]:
W_, b = CBM(W, Y_train, 1, 10001)

In [10]:
jll = predict(X_valid, W_, b)

In [48]:
scores = perplexity(jll, q_valid)

407	
6937	
1655	
224	
1573	
6613	
8345	
8618	
297	
9494	
9085	
2261	
7107	
5195	
8009	
5554	
6273	
922	
7771	
5130	
2408	
9824	
660	
8177	
5070	
9016	
5779	
1460	
4601	
1555	
4583	
8021	
5839	
5433	
2645	
5701	
5699	
8434	
4013	
7236	
5	
6	
7391	
3700	
8474	
2839	
7576	
844	


6710	
4194	
6409	
1971	
9521	
2053	
4732	
2975	
2611	
5007	
8220	
568	
5776	
9726	
4543	
7588	
6918	
3202	
8849	
4095	
6823	
7029	
6559	
202	
2837	
8379	
3429	
7196	
4074	
3433	
5862	
5177	
157	
4181	
706	
8420	
6646	
6425	
4001	
9541	
9158	
1017	
2451	
3556	
8510	
7129	
5301	
6969	
2190	
221	
8803	
6594	
500	
305	
8425	
5919	
4090	
4981	
8611	
8244	
4804	
2009	
9910	
471	
6	
9904	
1198	
514	
1061	
5049	
6289	
8585	
1697	
4648	
3676	
7371	
1867	
4857	
3924	
6939	
8431	
3086	
4288	
3138	
4486	
3949	
3730	
8607	
7310	
9582	
9873	
6426	
7482	
1215	
6676	
9535	
1417	
6268	
7276	
1861	
4232	
7448	
890	
93	
3984	
9691	
1013	
2390	
8062	
6904	
3130	
5344	
5758	
8122	
5480	
8897	
8612	
493	
3481	
8531	
8918	
3959	
1451	
9955	
4346	


8273	
6588	
623	
1684	
3113	
4831	
4864	
2999	
1296	
5455	
6438	
1174	
2614	
8980	
8335	
4497	
4971	
5907	
4306	
9466	
7984	
1854	
9071	
6252	
44	
6881	
7537	
9799	
3773	
5006	
2476	
4407	
5199	
23	
4765	
7780	
3844	
6157	
6404	
5952	
4183	
7238	
9997	
9110	
3673	
5611	
8292	
9821	
2692	
7749	
6771	
3543	
8974	
7290	
2434	
3308	
7711	
9445	
7937	
2	
3113	
7313	
9454	
6866	
143	
4840	
1922	
6999	
2972	
8987	
1629	
3123	
9338	
9287	
6150	
6167	
4133	
9119	
4102	
3205	
1107	
4185	
8085	
3128	
7953	
6895	
2502	
6291	
5583	
4604	
4278	
1887	
7569	
5986	
4312	
4874	
3322	
8738	
1715	
2	
3486	
2049	
1789	
3256	
3551	
4284	
3063	
9567	
1937	
3291	
2877	
5154	
5674	
1646	
2075	
6247	
9216	
3839	
8548	
9907	
1236	


6312	
7684	
796	
9586	
142	
8013	
76	
3413	
6459	
5376	
2202	
7398	
6192	
9011	
6426	
8628	
6651	
6580	
3281	
9037	
8311	
5611	
3377	
4726	
8609	
567	
6833	
4090	
4865	
7470	
7302	
661	
6275	
5051	
61	
8	
1022	
2020	
3498	
4045	
8883	
5416	
4154	
8307	
5096	
7597	
8776	
8903	
5411	
7769	
8427	
5058	
3804	
4666	
2107	
995	
5016	
770	
504	
908	
951	
572	
4840	
1995	
3133	
1515	
3947	
1118	
9237	
6711	
5420	
1351	
6495	
8244	
9260	
5839	
9350	
6276	
792	
4395	
1424	
3619	
544	
4855	
9062	
1404	
3414	
8250	
3180	
7133	
1076	
348	
2657	
4923	
5114	
8631	
1249	
7206	
2539	
9688	
3046	
3793	
5790	
9977	
2262	
9411	
4899	
9075	
6650	
8346	
8386	


661	
5045	
5149	
7027	
6442	
5060	
4505	
1992	
9206	
9220	
943	
1430	
6420	
5132	
6154	
1195	
5022	
163	
2033	
3408	
8900	
8684	
4354	
2506	
6872	
7930	
3656	
6620	
3054	
3546	
1922	
4786	
8013	
5228	
218	
677	
6045	
2960	
6883	
9379	
2051	
6739	
24	
6968	
1351	
8239	
7375	
5917	
2072	
566	
3318	
4445	
1660	
1379	
8672	
9006	
4336	
1284	
9524	
1427	
7906	
455	
1227	
546	
9030	
9202	
6286	
4046	
8540	
3644	
6759	
3251	
1451	
1108	
8911	
2648	
8547	
9401	
8637	
3907	
1375	
2299	
4070	
4598	
7	
6279	
3706	
1606	
34	
1747	
4094	
8670	
6418	
142	
8157	
5254	
7418	
9528	
615	
4788	
344	
7823	
7909	
7580	
9625	
3432	
4815	
4574	
4122	
2177	


...


6223	
4051	
2641	
3099	
460	
7462	
4280	
4708	
2101	
5176	
280	
7383	
1636	
6612	
3799	
1260	
2251	
8563	
1321	
7057	
2410	
6082	
351	
990	
4401	
696	
4991	
1759	
6155	
9290	
2814	
8781	
6207	
3481	
8199	
4351	
4266	
3632	
6868	
3685	
5235	
4733	
8079	
1019	
1634	
646	
1087	
7808	
5738	
3151	
7114	
1407	
4030	
5185	
4364	
9396	
2411	
7458	
6531	
3633	
5386	
272	
188	
9340	
6427	
2676	
6281	
8778	
5459	
4348	
5053	
8532	
7524	
4121	
1203	
578	
1691	
4020	
4384	
4559	
8115	
2	
6736	
6593	
1481	
5136	
2500	
4546	
7627	
8020	
124	
4883	
1356	
8217	
2236	
3946	
6731	
1575	
3443	
608	
7997	
4967	
1312	
6751	
6271	
8774	
799	
6905	
5087	
8890	
5045	
6178	
6776	
4897	
4467	
1105	
10	
6573	
3720	
5293	
3332	
6933	
5697	
4063	
29	
908	
4723	


8360	
184	
9887	
1627	
3129	
4269	
6731	
2965	
1250	
882	
4355	
2778	
6758	
7784	
8252	
4171	
6951	
4856	
3207	
918	
2695	
1772	
1672	
9838	
6289	
5915	
5126	
6773	
168	
3189	
1285	
4508	
879	
1514	
4801	
4685	
8551	
2693	
9723	
8023	
4778	
1592	
7090	
3605	
3100	
5587	
7099	
7860	
7064	
3877	
1304	
3178	
9266	
8763	
7489	
117	
9570	
7805	
5951	
4336	
6721	
6711	
3235	
6529	
8348	
9263	
4692	
429	
7945	
1515	
1201	
1874	
5842	
848	
5871	
2220	
2230	
9666	
4331	
7235	
714	
8610	
4998	
9518	
263	
1736	
9540	
9399	
3127	
6178	
7356	
6505	
6461	
9541	
19	
1932	
8298	
3782	
4753	
4320	
1228	
1122	
9914	
8028	
4277	
7263	
9857	
9305	
8528	
9119	
3511	
9583	
8927	
2964	
5499	
7735	
1723	
6335	
2749	
4595	
5183	
2480	
5869	
3513	
4914	
708	
4555	


5961	
788	
805	
2039	
1575	
3199	
4767	
8007	
8494	
1253	
7207	
6681	
9607	
9702	
1153	
9035	
6500	
3764	
7083	
8536	
4860	
9077	
6683	
5788	
9530	
3689	
5023	
9783	
4748	
953	
6286	
8578	
8009	
5223	
8958	
1924	
2230	
4491	
7756	
209	
4311	
8371	
147	
5474	
4220	
8592	
6963	
7554	
7865	
6517	
8097	
7914	
5177	
7266	
5266	
221	
6260	
6886	
3655	
7224	
4029	
7407	
8308	
5771	
7967	
8056	
5593	
3568	
1528	
7524	
4751	
9355	
5907	
2284	
7867	
9133	
2819	
4953	
2246	
2667	
3999	
5912	
3201	
150	
2374	
7398	
2280	
5346	
9466	
6090	
4828	
3161	
8500	
5135	
3462	
353	
8709	
7859	
5486	
8426	
1780	
9418	
1223	
923	
1086	
9966	
202	
6103	
9898	
5237	
88	
5509	
6310	
6940	
7480	
9037	
3898	
423	
60	
121	
6783	
6407	
3924	
8827	
7842	
9823	
8713	
4921	
2296	
2435	
8964	


2526	
9360	
1769	
3965	
807	
5598	
4622	
7460	
9491	
1691	
2016	
4734	
8414	
2130	
5900	
4383	
7360	
8960	
6012	
8683	
2944	
4966	
1442	
8166	
4936	
2700	
3100	
2024	
7272	
448	
8402	
6649	
530	
3747	
1189	
975	
1086	
10001	


bad argument #2 to '?' (out of range at /tmp/luarocks_torch-scm-1-8597/torch7/generic/Tensor.c:888)
stack traceback:
	[C]: at 0x0b3341c0
	[C]: in function '__index'
	[string "function perplexity(jll, Y)..."]:7: in function 'perplexity'
	[string "scores = perplexity(jll, q_valid)..."]:1: in main chunk
	[C]: in function 'xpcall'
	/Users/Vincent/torch/install/share/lua/5.1/itorch/main.lua:179: in function </Users/Vincent/torch/install/share/lua/5.1/itorch/main.lua:143>
	/Users/Vincent/torch/install/share/lua/5.1/lzmq/poller.lua:75: in function 'poll'
	...s/Vincent/torch/install/share/lua/5.1/lzmq/impl/loop.lua:307: in function 'poll'
	...s/Vincent/torch/install/share/lua/5.1/lzmq/impl/loop.lua:325: in function 'sleep_ex'
	...s/Vincent/torch/install/share/lua/5.1/lzmq/impl/loop.lua:370: in function 'start'
	/Users/Vincent/torch/install/share/lua/5.1/itorch/main.lua:350: in main chunk
	[C]: in function 'require'
	(command line):1: in main chunk
	[C]: at 0x010b127bd0: 

In [46]:
q_valid:size()

 3370
   50
[torch.LongStorage of size 2]

